In [17]:
let rnd = new Random();

let random_number =
    fun min max ->  int (rnd.NextDouble() * (max - min) + min)

let random_number_std =
    fun () -> random_number -5.0 10.0

let random_step = 
    fun () -> match rnd.Next (1, 3) with
                    | 0 -> 0.0
                    | 1 -> -0.1
                    | 2 -> 0.1
let random_bool =
    fun () -> rnd.NextDouble() >= 0.5;

let shuffle d =
    d |> List.sortBy(fun _ -> rnd.Next(1, (List.length d)) )

In [18]:
// p1 = f1 / (f1 + f2 + f3 + f4)

type Parameters = {
    A: float
    B: float
    C: float
    D: float
    E: float
    F: float
}

// Curvefitting algorithm
let curvefitting_algo (x: float) (p: Parameters) = 
    p.A*x**5 + p.B*x**4 + p.C*x**3 + p.D*x**2 + p.E*x + p.F

// Curvefitting algorithm prefilled with the correct values.
let check_curvefitting_algo x =
    { A = -0.2; B = 3.0; C = -5.6; D = 8.0; E = -5.0; F = -5.0; } |> curvefitting_algo x

let y_points = [for x in -5.0..5.0 do check_curvefitting_algo x]

let calculate_fitness parameters =
    let y_cap_points = [for x in -5.0..5.0 do (curvefitting_algo x parameters)]
    ((List.sum y_points)-(List.sum y_cap_points))**2

let calculate_fitness_for_list parameters =
    (calculate_fitness parameters), parameters

let mutate_float f_in =
    f_in + (random_step ())

let sort_list_by_fitness list =
        list |>
        List.map (calculate_fitness_for_list) |>
        List.sortBy (fun (fitness_score, parameters) -> fitness_score)

//Random Wheel Selection. Based on https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
let rwc (source: list<Parameters>) =
    let source_fitness = source |> List.map (fun parameters -> calculate_fitness parameters)
    let max =  source_fitness |> List.sum
    let pick = random_number 0 max
    let mutable cum_fitness = 0.0
    match source_fitness |> List.zip source |> List.tryPick (fun (parameters, fitness) -> (
        cum_fitness <- cum_fitness + fitness
        if cum_fitness > pick then
            cum_fitness <- 0
            Some (parameters)
        else
            None
        )) with
        | Some(value) -> value
        | None -> failwith "Error: no input list"

// Get two children
let get_child parenta parentb =
    let get_child_func a b = { Parameters.A = mutate_float a.A; B = mutate_float a.B; C = mutate_float a.C; D = mutate_float b.D; E = mutate_float b.E; F = mutate_float b.F }
    match (random_bool ()) with
        | true -> get_child_func parenta parentb
        | false -> get_child_func parentb parenta

let rec reproduction amount_of_children (children: list<Parameters>) (parents: list<Parameters>) =
    match amount_of_children with
        | amount_of_children when (amount_of_children > 0) -> reproduction (amount_of_children - 1) ((get_child (rwc parents) (rwc parents)) :: children) parents
        | _ -> children

In [19]:
let random_organisms num_of_organisms = [for i in 0..num_of_organisms do {Parameters.A = random_number_std (); B = random_number_std (); C = random_number_std (); D = random_number_std (); E = random_number_std (); F = random_number_std () }]

let epoch_prep num_of_organisms list = 
        list |>
        sort_list_by_fitness |>
        List.map (fun (_, parameters) -> parameters)
        // List.take num_of_organisms |>
let epoch num_of_organisms organisms =
        organisms |>
        epoch_prep num_of_organisms |>
        // Simple diversification algorithm: 80% reproduced, 20% random.
        reproduction ((num_of_organisms/10) * 8) [] |>
        (fun reproduced -> random_organisms ((num_of_organisms/10) * 2) @ reproduced)
    
let rec geneticAlgorithm n_epochs num_of_organisms generation =
        match (n_epochs) with
                | 0 -> generation
                | 1 -> generation |> epoch_prep num_of_organisms
                | n -> geneticAlgorithm (n-1) num_of_organisms (epoch num_of_organisms generation)

let num_of_organisms_input = 180
let num_of_epochs = 50
let input = random_organisms num_of_organisms_input
printfn "Input: %A" (input |> sort_list_by_fitness).Head
let bestAlgo = (geneticAlgorithm num_of_epochs num_of_organisms_input input).Head
bestAlgo |> calculate_fitness_for_list

//Actual parameters: { A = -0.2; B = 3.0; C = -5.6; D = 8.0; E = -5.0; F = -5.0; }

Input: (1089.0, { A = -2.0
           B = 3.0
           C = -1.0
           D = 8.0
           E = -3.0
           F = -2.0 })


(1.2099999999967985, { A = 9.1\n B = 3.1\n C = -1.1\n D = 5.9\n E = 1.7\n F = -1.9 }) Item1 1.2099999999967985 Item2 { A = 9.1\n B = 3.1\n C = -1.1\n D = 5.9\n E = 1.7\n F = -1.9 } A 9.1 B 3.1 C -1.1 D 5.9 E 1.6999999999999997 F -1.899999999999999